In [43]:
#首先导入所需的库
import xml.etree.cElementTree as ET
from collections import Counter
from collections import defaultdict
import re
import csv
import codecs
import pprint
#import schema     
#import cerberus

In [22]:
#定义一个函数count_tags，返回数据集中的所有标签及其个数
def count_tags(filename):
    tags = (el.tag for _, el in ET.iterparse(filename))
    return Counter(tags)

In [4]:
count_tags('export.osm')

Counter({'member': 30408,
         'meta': 1,
         'nd': 318023,
         'node': 251925,
         'note': 1,
         'osm': 1,
         'relation': 673,
         'tag': 90225,
         'way': 32974})

In [23]:
#定义一个函数count_type，返回一个字典，键为标签'tag'当中k值的四种类型，分为全部小写、带冒号、存在有问题字符和其他，值为每种类型的个数。
lower = re.compile(r'^([a-z]|_)*$')  #将正则表达式的字符串形式编译为Pattern实例
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        k=element.get('k')  #获取属性k的值，这里也可以用element.attrib['k']
        if lower.match(k): #match，从开头就符合正则
            keys["lower"]+=1
        elif lower_colon.search(k): #search，只要字符串里面有符合正则的
            keys['lower_colon']+=1
        elif problemchars.search(k):
            keys['problemchars']+=1
        else:
            keys['other']+=1
    
    return keys

def count_type(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [6]:
count_type('export.osm')

{'lower': 74783, 'lower_colon': 13332, 'other': 2110, 'problemchars': 0}

In [26]:
#定义一个函数find_uid，返回向该数据集作出贡献的唯一用户
def find_uid(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.tag=='node' or element.tag=='way' or element.tag=='relation':
            users.add(element.get('uid')) #这里无需将值整数化，因为得到的本来就是整数而非字符串

    return users

In [8]:
find_uid('export.osm')

{'10023',
 '100508',
 '1016084',
 '103253',
 '103866',
 '1051550',
 '1061602',
 '106914',
 '1069176',
 '1081635',
 '111462',
 '114',
 '1146641',
 '116306',
 '1164',
 '120556',
 '1219875',
 '1222784',
 '1226694',
 '1229704',
 '1230673',
 '1231097',
 '123364',
 '1236135',
 '1240849',
 '1240864',
 '1242214',
 '1265238',
 '128186',
 '129814',
 '131968',
 '1334329',
 '1362927',
 '1371102',
 '1431737',
 '145231',
 '1486839',
 '150368',
 '1530720',
 '1549003',
 '1553228',
 '155542',
 '1577332',
 '1595392',
 '160949',
 '161619',
 '1632621',
 '1640335',
 '1642501',
 '1651798',
 '16558',
 '1656925',
 '165869',
 '1673659',
 '1679',
 '169827',
 '1707971',
 '1723427',
 '1726068',
 '1732925',
 '1734094',
 '1747411',
 '1749066',
 '17497',
 '1764612',
 '1766553',
 '1784545',
 '1787380',
 '18069',
 '181561',
 '1817244',
 '1828980',
 '1829683',
 '1832122',
 '1853207',
 '1860786',
 '1870014',
 '1872850',
 '1877031',
 '1879520',
 '1903826',
 '190389',
 '1911765',
 '1928564',
 '1931010',
 '1934529',
 '1951

In [34]:
#定义一个函数audit，返回一个字典，键是所有的街道类型，值是各街道类型对应的街道名集合
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m :
        street_type = m.group()
        street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if (elem.tag=='tag') and is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])
    osm_file.close()            
    return street_types

In [35]:
#将结果以一一对应的形式打出来观察
for m,n in count_street_type('export.osm').items():
    for i in n:
        print m,i

Boulevard 北环大道 Beihuan Boulevard
Boulevard 深南大道 Shennan Boulevard
Boulevard Shennan Boulevard
Boulevard 滨河大道 Binhe Boulevard
Boulevard 南山大道 Nanshan Boulevard
District Xili Town, Nanshan District
1號 金鸡路1號
Rd 正云路 Zhengyun Rd
Rd Futianxinsha Rd
Rd 玉园路 Yuyuan Rd
Rd Guihua Rd
Rd 正风路 Zhengfeng Rd
Rd 罗芳路 Luofang Rd
Rd 延芳路 Yanfang Rd
Rd 景田北街 Jingtian N Rd
Rd 经二路 Jinger Rd
Rd Gongye 7th Rd
Rd 湖贝路 Hubei Rd
Rd 莲花路 Lianhua Rd
Rd 香梅路 Xiangmei Rd
Rd 布龙路 Bulong Rd
Rd 中山园路 Zhongshanyuan Rd
Rd Zhenghua Rd
Rd 中兴路 Zhongxing Rd
Rd 景田西路 Jingtian W Rd
Rd 景田路 Jingtian Rd
Rd 粤兴三道 Yuexing 3rd Rd
Rd 东门中路 Dongmen Middle Rd
Rd 文锦中路 Wenjin Middle Rd
Rd 新秀路 Xinxiu Rd
Rd 欣园路 Xinyuan Rd
Tin 青山公路 - 新田段 Castle Peak Road - San Tin
Lu 沙河西路 ShaHe Xi Lu
Lu KeYuan Lu
Lu Hua Fa Bei Lu
Lu 桃园路 Taoyuan Lu
Lu Hongbao Lu
S. Gaoxin S.
BLVD Shennan BLVD
Xinsha Xinsha
CoCoPark 星河CoCoPark
Highway 广深公路 Guangzhou-Shenzhen Highway
B座2樓 海岸城中洲控股金融中心B座2樓
518000 1002 Huaqiang N Rd, HuaQiang Bei, Futian Qu, Shenzhen Shi, Guangdong Sheng, Chi

In [44]:
#expected列表包含了我们所期望的规范化模板。
expected = ["Street", "Avenue", "Boulevard", "Road", 'District', 'Highway', 'Path']

#根据上面的结果，建立mapping字典，键是变更前的街道类型，值是变更后的街道类型，这里只修改英文字符，且对非街道类型不做修改。
mapping = { 'S':"Street",
            'S.':"Street",
            "St": "Street",   
            'Lu':'Road',
            'Rd':"Road",
            'road':'Road',
            'BLVD':"Boulevard",
            'DaDao':'Road',
            'Av':"Avenue",
            'Ave':"Avenue"
            }

def add_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

#替换街道名中的街道类型
def update_name(name, mapping):
    shortname = mapping.keys()
    for word in shortname:
        if name.endswith(word):
            name = name.replace(word,mapping[word])
    return name

In [39]:
#呈现变更的成果
st_types = audit('export.osm')
for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name

北环大道 Beihuan Boulevard => 北环大道 Beihuan Boulevard
深南大道 Shennan Boulevard => 深南大道 Shennan Boulevard
Shennan Boulevard => Shennan Boulevard
滨河大道 Binhe Boulevard => 滨河大道 Binhe Boulevard
南山大道 Nanshan Boulevard => 南山大道 Nanshan Boulevard
Xili Town, Nanshan District => Xili Town, Nanshan District
金鸡路1號 => 金鸡路1號
正云路 Zhengyun Rd => 正云路 Zhengyun Road
Futianxinsha Rd => Futianxinsha Road
玉园路 Yuyuan Rd => 玉园路 Yuyuan Road
Guihua Rd => Guihua Road
正风路 Zhengfeng Rd => 正风路 Zhengfeng Road
罗芳路 Luofang Rd => 罗芳路 Luofang Road
延芳路 Yanfang Rd => 延芳路 Yanfang Road
景田北街 Jingtian N Rd => 景田北街 Jingtian N Road
经二路 Jinger Rd => 经二路 Jinger Road
Gongye 7th Rd => Gongye 7th Road
湖贝路 Hubei Rd => 湖贝路 Hubei Road
莲花路 Lianhua Rd => 莲花路 Lianhua Road
香梅路 Xiangmei Rd => 香梅路 Xiangmei Road
布龙路 Bulong Rd => 布龙路 Bulong Road
中山园路 Zhongshanyuan Rd => 中山园路 Zhongshanyuan Road
Zhenghua Rd => Zhenghua Road
中兴路 Zhongxing Rd => 中兴路 Zhongxing Road
景田西路 Jingtian W Rd => 景田西路 Jingtian W Road
景田路 Jingtian Rd => 景田路 Jingtian Road
粤兴三道 Yuexing

In [40]:
#定义函数shape_element，将XML的元素从文档形式转换为能够写入CSV的数据结构

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags=[]
    m=0  #要在for child in element:循环之前规定，否则每次一循环就归零了
    # YOUR CODE HERE
    if element.tag == 'node':
        for i in NODE_FIELDS:
            node_attribs[i]=element.get(i)
        if element.find('tag') is not None:
            for tag in element.iter('tag'):  #iter生成迭代来查询
                tag_dict={}   #定义要在for循环里面，因为如果是字典嵌套在列表里，每个字典又有共同的key,定义在外面会导致字典的值都是最后一次循环的，详细参考word
                if PROBLEMCHARS.search(tag.get('k')): #如果k值存在问题
                    continue   #即当满足该条件时跳过，继续循环，这里也可以用‘pass’
                else:  #如果k值不存在问题
                    tag_dict['id']=element.get('id')
                    tag_dict['value']=tag.get('v')
                    if LOWER_COLON.search(tag.attrib['k']): #如果k值存在冒号
                        tag_dict['key']=tag.get('k').split(':',1)[1] #以冒号来分隔一次，取第二个元素，PS：split是返回分隔后字符串的列表
                        tag_dict['type']=tag.get('k').split(':',1)[0]
                    else: #如果k值不存在冒号
                        tag_dict['key']=tag.get('k')
                        tag_dict['type']='regular'
                tags.append(tag_dict)
        else:
            pass  #如果不存在次级标记，就啥也不做，即tags还是空列表
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for p in WAY_FIELDS:
            way_attribs[p]=element.get(p)
        for child in element:
            if child.tag=='tag':
                tag_dict={}
                if PROBLEMCHARS.search(child.get('k')):
                    continue
                else:
                    tag_dict['id']=element.get('id')
                    tag_dict['value']=child.get('v')
                    if LOWER_COLON.search(child.attrib['k']):
                        tag_dict['key']=child.get('k').split(':',1)[1]
                        tag_dict['type']=child.get('k').split(':',1)[0]
                    else:
                        tag_dict['key']=child.get('k')
                        tag_dict['type']='regular'
                tags.append(tag_dict)
            elif child.tag is None:
                pass
            elif child.tag=='nd':
                way_dict={}
                way_dict['id']=element.get('id')
                way_dict['node_id']=child.get('ref')
                way_dict['position']=m
                m+=1
                way_nodes.append(way_dict)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

In [42]:
#将每个数据结构写入相应的 .csv 文件

#SCHEMA = schema.schema

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"


def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            
#因为课程中的习题已经做过验证了，这里不再重复
#def validate_element(element, validator, schema=SCHEMA):
#    """Raise ValidationError if element does not match schema"""
#    if validator.validate(element, schema) is not True:
#         field, errors = next(validator.errors.iteritems())
#         message_string = "\nElement of type '{0}' has the following errors:\n{1}"
#         error_string = pprint.pformat(errors)
        
#        raise Exception(message_string.format(field, error_string))
        
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
            
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()
        
       #validator = cerberus.Validator()


        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
               #if validate is True:
                    #validate_element(el, validator)
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
                    
if __name__ == '__main__':
    process_map('export.osm',validate =False)
            